In [1]:
from math import factorial
import sys, os, random, copy, math, pickle, visualize

import matplotlib.pyplot as plt
import numpy as np

from deap import algorithms
from deap import base
from deap.benchmarks.tools import igd
from deap import creator
from deap import tools

sys.path.insert(0, 'evoman') 
from environment import Environment
from demo_controller import player_controller

# choose this for not using visuals and thus making experiments faster
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

# Init experiment
experiment_name = 'deap_generalized_agent'
OUTPUT_DIR = './' + experiment_name + '/'
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

# global training parameters
n_population = 100
n_generations = 250
min_value = -1
max_value = 1
# Neural net parameters
n_hidden_neurons = 10
    # initializes environment with ai player using random controller, playing against static enemy
env = Environment(experiment_name=experiment_name,
                  multiplemode="yes",
                  randomini="yes",
                  clockprec="medium",
                  enemies=[1, 2, 3, 4, 5, 6, 7, 8],
                  playermode="ai",
                  player_controller=player_controller(n_hidden_neurons),
                  enemymode="static",
                  level=2,
                  speed="normal")
n_weights = (env.get_num_sensors()+1)*n_hidden_neurons+(n_hidden_neurons+1)*5
# DEAP parameters
n_objectives = 3
n_partitions = 12

def eval_individual(individual):
    """Function to evaluate current individual """
    fitness, player_life, enemy_life, time = env.play(pcont=individual)
    x1 = fitness
    x2 = (player_life - enemy_life) - np.log(time)
    x3 = player_life - np.log(time)
    print((x1, x2, x3))
    return (x1, x2, x3)

def get_pickle_object(file_path):
    file = open(file_path,'rb')
    object_file = pickle.load(file)
    return object_file

# Create uniform reference point
ref_points = tools.uniform_reference_points(n_objectives, n_partitions)

# Create classes
creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0, 1.0))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

# Toolbox initialization
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.uniform, min_value, max_value)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=n_weights)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", eval_individual)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=min_value, up=max_value, eta=30.0)
toolbox.register("mutate", tools.mutPolynomialBounded, low=min_value, up=max_value, eta=20.0, indpb=1.0/n_weights)
toolbox.register("select", tools.selNSGA3, ref_points=ref_points)

pygame 2.1.2 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html

MESSAGE: Pygame initialized for simulation.


In [ ]:
test = get_pickle_object('./deap_generalized_agent/deap_stats_object')

print(test)

In [6]:
hof = 0
hof_tup = (0.0, 0.0, 0.0)
for key, value in test.items():
    if type(key) == int and key >= 264:
        for individual in value:
            if individual.fitness.values[1] > hof_tup[1]:
                hof = individual
                hof_tup = individual.fitness.values
                print(individual.fitness.values)

(91.10669517527555, 58.70669517527585, 58.70669517527585)
(90.92661051127251, 59.3016826334523, 59.3016826334523)


In [7]:
fitness, player_life, enemy_life, time = env.play(pcont=hof)


RUN: run status: enemy: 1; fitness: 4.456705217729996; player life: 0; enemy life: 90.0; time: 94

RUN: run status: enemy: 2; fitness: 91.35657488308075; player life: 64; enemy life: 0; time: 155

RUN: run status: enemy: 3; fitness: 49.02326625757942; player life: 0; enemy life: 40.0; time: 145

RUN: run status: enemy: 4; fitness: 21.82385026742617; player life: 0; enemy life: 70.0; time: 177

RUN: run status: enemy: 5; fitness: 87.05520676758503; player life: 31.600000000000264; enemy life: 0; time: 448

RUN: run status: enemy: 6; fitness: 57.5489615464343; player life: 0; enemy life: 30.0; time: 233

RUN: run status: enemy: 7; fitness: 89.43679627126204; player life: 47.2000000000003; enemy life: 0; time: 197

RUN: run status: enemy: 8; fitness: 85.83531660910582; player life: 19.600000000000236; enemy life: 0; time: 457
